In [ ]:
from collections import namedtuple
import numpy as np
import nest
from scaffold.core import from_hdf5
from scaffold.config import JSONConfig
from scaffold.output import HDF5Formatter
from population_view import PopView
from world_populations import Planner, Cortex, SensoryIO, MotorIO, DirectDCN, InverseDCN

In [ ]:
CORES = 8

nest.Install("extracerebmodule")

nest.set_verbosity('M_WARNING')
nest.ResetKernel()
nest.SetKernelStatus({'local_num_threads' : CORES,
                      'total_num_virtual_procs' : CORES,
                      'resolution' : 1.0,
                      'overwrite_files' : True})

In [ ]:
trial_len = 300

In [ ]:
def create_brain(prism):
    hdf5_file = 'mouse_cerebellum_200.hdf5'
    # hdf5_file = 'scaffold_network_VOR.hdf5'
    # json_file = 'scaffold_network_VOR_10.json'
    
    # reconfigured_obj = JSONConfig(json_file)
    # HDF5Formatter.reconfigure(hdf5_file, reconfigured_obj)
    
    scaffold_model = from_hdf5(hdf5_file)
    scaffold_model.configuration.verbosity = 3

    # Get scaffold model populations
    S_MF = scaffold_model.get_entities_by_type("mossy_fibers")
    S_IO = scaffold_model.get_cells_by_type("io_cell")[:, 0]
    S_DCN = scaffold_model.get_cells_by_type("dcn_cell")[:, 0]

    uz_pos = scaffold_model.labels["microzone-positive"]
    uz_neg = scaffold_model.labels["microzone-negative"]

    S_DCNp = np.intersect1d(S_DCN, uz_pos)
    S_DCNn = np.intersect1d(S_DCN, uz_neg)
    S_IOp = np.intersect1d(S_IO, uz_pos)
    S_IOn = np.intersect1d(S_IO, uz_neg)
    #

    # Prepare adapters
    adapter_forward = scaffold_model.create_adapter("FCN_2019")
    adapter_inverse = scaffold_model.create_adapter("FCN_2019")

    adapter_forward.enable_multi("forward")
    adapter_inverse.enable_multi("reverse")

    adapter_forward.prepare()
    adapter_inverse.prepare()
    #

    # Get NEST populations
    f_IOp = adapter_forward.get_nest_ids(S_IOp)
    f_IOn = adapter_forward.get_nest_ids(S_IOn)

    i_IOp = adapter_inverse.get_nest_ids(S_IOp)
    i_IOn = adapter_inverse.get_nest_ids(S_IOn)

    f_DCNp = adapter_forward.get_nest_ids(S_DCNp)
    f_DCNn = adapter_forward.get_nest_ids(S_DCNn)

    i_DCNp = adapter_forward.get_nest_ids(S_DCNp)
    i_DCNn = adapter_forward.get_nest_ids(S_DCNn)

    f_MF = adapter_forward.get_nest_ids(S_MF)
    i_MF = adapter_inverse.get_nest_ids(S_MF)
    #

    # Define population views
    MF_number = len(f_MF)
    IO_number = len(f_IOp) + len(f_IOn)

    planner_pv = Planner(MF_number, prism)
    cortex_pv = Cortex(MF_number)

    f_IO_pv = SensoryIO(f_IOp, f_IOn)    i_IO_pv = MotorIO(i_IOp, i_IOn)
    f_DCN_pv = DirectDCN(f_DCNp, f_DCNn)
    i_DCN_pv = InverseDCN(i_DCNp, i_DCNn)

    f_MF_pv = PopView(f_MF)
    i_MF_pv = PopView(i_MF)
    #


    planner_pv.connect(cortex_pv)

    cortex_pv.connect(f_MF_pv)  # Efference copy
    planner_pv.connect(i_MF_pv)  # Sensory input

    conn_dict = {"rule": "fixed_indegree", "indegree": 1}
    nest.Connect(f_DCNp, cortex_pv.pop, conn_dict, {'weight': 1.0})
    nest.Connect(f_DCNn, cortex_pv.pop, conn_dict, {'weight': -1.0})

    # TODO: altre connesisoni?
    return cortex_pv

In [ ]:
prism = 25.0

cortex = create_brain(prism)
from time import time

t0 = time()
nest.Simulate(trial_len)
t1 = time()

print(t1-t0)

In [ ]:
cortex.plot_spikes()